## Equity analysis, comparing all models for one store placement

### High level process:
For each model:
- Take the optimum store placement
- Find the new buildings that got access from the new store. Find which census tracts they fall in. Those are census tracts with new access
- Find the buildings that still don't have access even after the new store placement. Those are census tracts without new access
- Aggregate and compare demographic information between these 2 groups of census tracts, through a weighted aggregation method where the weight is their population

### Implementation:
- Part 1: Get the demographic information, construct and merge on accurate tract ids (geoID)
- Part 2: For each model, find and store which census tracts their optimum store location gives access to
- Part 3: Conduct equity comparison calculations for each model


In [23]:
# Import libraries
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import haversine as hs
import gurobipy as gp
from gurobipy import GRB
from shapely import wkt

# Helper modules
import helper_population_allocation as pa
import helper_distance_calculation as dc

# Avoid printing set copy warnings
import warnings
warnings.filterwarnings("ignore")

## Prepare demographic information

In [24]:
# Importing files
usda = pd.read_csv('../input_data/usda_lowincomelowaccess.csv') # Entire USDA dataset with indicator denoting whether a tract is low income low access
equity = pd.read_csv('../input_data/Demographics_Combined_Updated.csv') # Demographic information

# Merge with demographics dataset to get correct tract ids
usda.rename(columns = {'Allegheny_Tracts_GEOID':'GEOID'}, inplace = True)
equity.rename(columns = {'CensusTract':'GEOID'}, inplace = True)
merged = pd.merge(usda, equity, on="GEOID")

# Compute additional columns for equity analysis
merged["PctSeniorPop"] = round((merged["Senior"] / merged["USDA_Data_Pop2010"])*100,2)  
merged["PctSNAPHH"] = round((merged["SNAP"] / merged["USDA_Data_OHU2010"])*100, 2)
merged["PctSingleHH"] = round((merged["SingleHH"] / merged["USDA_Data_OHU2010"])*100, 2)
merged["PctNoVehicleHH"] = round((merged["NoVehicle"] / merged["USDA_Data_OHU2010"])*100, 2)
merged["PctDisabilityPop"] = round((merged["Disability"] / merged["USDA_Data_Pop2010"])*100, 2)
merged["Race"] = merged["Race"]/100
merged["Black_Pop"] = round(merged["Race"]*merged["USDA_Data_Pop2010"],0)
merged["USDA_Data_PovertyRate"] = merged["USDA_Data_PovertyRate"] / 100

merged.head(5)

,OBJECTID,Allegheny_Tracts_FID_1,Allegheny_Tracts_STATEFP,Allegheny_Tracts_COUNTYFP,Allegheny_Tracts_TRACTCE,Allegheny_Tracts_AFFGEOID,GEOID,Allegheny_Tracts_NAME,Allegheny_Tracts_LSAD,Allegheny_Tracts_ALAND,...,NoVehicle,Disability,Race,SingleHH,PctSeniorPop,PctSNAPHH,PctSingleHH,PctNoVehicleHH,PctDisabilityPop,Black_Pop
0,1,1,42,3,50900,1400000US42003050900,42003050900,509.0,CT,452643,...,650.0,264.0,0.927,184.0,14.89,70.66,28.26,99.85,21.96,1114.0
1,2,2,42,3,70300,1400000US42003070300,42003070300,703.0,CT,428546,...,146.0,153.0,0.034,75.0,14.79,1.61,6.37,12.39,6.96,75.0
2,3,3,42,3,120700,1400000US42003120700,42003120700,1207.0,CT,514927,...,127.0,NaN,NaN,NaN,24.57,37.44,NaN,30.68,NaN,NaN
3,4,4,42,3,140400,1400000US42003140400,42003140400,1404.0,CT,834995,...,50.0,111.0,0.017,62.0,15.61,1.16,6.54,5.27,4.81,39.0
4,5,5,42,3,180700,1400000US42003180700,42003180700,1807.0,CT,393380,...,172.0,280.0,0.280,115.0,9.95,24.67,12.78,19.11,13.92,563.0


## Pre-process information from all the 3 models

1. Model 1 - Create arrays to track ordering
2. Model 2 - Get the chosen store manually and store it
3. Model 3 - Get the chosen store manually and store it

#### Model 1

In [25]:
# Get the main buildings dataset 
buildings_df = gpd.read_file('../processed_data/relevant_buildings.shp')

# Create ID variable
buildings_df.reset_index(drop=True, inplace=True)
buildings_df['building_id'] = buildings_df.index + 1
buildings_df['building_id'] = buildings_df.apply(lambda row: str(row['building_id']) + '-' + str(row['CLASS']) , axis=1)

# Matching GEOIDs
acs_data = pd.read_csv('../processed_data/ACSData.csv')
acs_data = acs_data[['geometry', 'geo_id','B01003_001E', 'B25010_001E']]
acs_data['geometry'] = acs_data['geometry'].apply(wkt.loads)
acs_data = gpd.GeoDataFrame(acs_data, geometry = 'geometry').set_crs(buildings_df.crs)

buildings_df = buildings_df.sjoin(acs_data, how="left", predicate = 'intersects')
buildings_df["geo_id"] = buildings_df["geo_id"]
buildings_df = buildings_df.drop_duplicates("building_id")

# Create arrays to track ordering (residential)
res_buildings = buildings_df[buildings_df['class_reco'].str.contains('Residential')]
res_buildings = res_buildings.sort_values('building_id')
res_buildings = dc.get_geocoordinate(res_buildings, 'geometry')

res_buildings_array = np.array(res_buildings['building_id'])
res_buildings_coordinates_array = np.array(res_buildings['coordinates'])

# Create arrays to track ordering (Commercial)
comm_buildings = buildings_df[buildings_df['class_reco'].str.contains('commercial')]
comm_buildings = comm_buildings.sort_values('building_id')
comm_buildings = dc.get_geocoordinate(comm_buildings, 'geometry')

comm_buildings_array = np.array(comm_buildings['building_id'])
comm_buildings_coordinates_array = np.array(comm_buildings['coordinates'])

# Create arrays to track ordering (grocery stores)
grocery_stores = buildings_df[buildings_df['class_reco'].str.contains('Grocery')]
grocery_stores = grocery_stores.sort_values('building_id')
grocery_stores = dc.get_geocoordinate(grocery_stores, 'geometry')

grocery_stores_array = np.array(grocery_stores['building_id'])  # ith element represents the building id of ith grocery store
grocery_stores_coordinates_array = np.array(grocery_stores['coordinates'])  # ith element represents the coordinates of the ith grocery store

#New Stores
chosen_stores = pd.read_csv('../processed_data/new_store_ids.csv')
chosen_store = chosen_stores['building_id'].iloc[0]

# Importing Distance Matrix
res_comm_distance_matrix = np.load('../processed_data/res_comm_distance_matrix.npy')

# Creating a res comm access matrix 
# [i,j] element indicates whether the ith residential building and jth commercial building are within 0.5 miles of each other
res_comm_access_matrix_half_mile = res_comm_distance_matrix.copy()
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile <= 0.5] = 1
res_comm_access_matrix_half_mile[res_comm_access_matrix_half_mile != 1] = 0

# Create parameter matrices (Res groc access array)
# ith value indicates whether the ith residential building has existing access
res_groc_distance_matrix, res_groc_access_matrix = dc.calculate_access(res_buildings_coordinates_array, grocery_stores_coordinates_array)
res_groc_access_matrix_half_mile = res_groc_distance_matrix.copy()
res_groc_access_matrix_half_mile[res_groc_access_matrix_half_mile <= 0.5] = 1
res_groc_access_matrix_half_mile[res_groc_access_matrix_half_mile != 1] = 0

res_access_array_half_mile = np.amax(res_groc_access_matrix_half_mile, 1)


In [26]:
# Find which residential buildings already have access, subset accordingly 
existing_access_indices = res_access_array_half_mile.nonzero()[0] # These are indices of residential buildings that currently have access
res_comm_access_matrix_subset = np.delete(res_comm_access_matrix_half_mile, existing_access_indices, axis=0 )

chosen_store_index = np.where(comm_buildings_array == chosen_store)[0][0]

#New residential buildings that will have access based on commercial building
new_building_with_access = res_comm_access_matrix_half_mile[:,2206]
new_building_with_access = new_building_with_access.nonzero()[0]
new_building_with_access = [i for i in new_building_with_access if i not in existing_access_indices]

# Get IDs of all residential buildings with access
existin_access_res_buildings = res_buildings_array[new_building_with_access]

new_access_buildings = buildings_df[buildings_df.building_id.isin(existin_access_res_buildings)]

# Census Tracts with New Access to Gracery Stores
census_tracts_with_new_access_m1 = set(new_access_buildings['geo_id'])

# Census tracts without access even after the store placement
original_buildings_without_access = np.where(res_access_array_half_mile==0)[0] #Buildings that dont have access
remaining_buildings_without_access = [i for i in original_buildings_without_access if i not in new_building_with_access]
no_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[remaining_buildings_without_access])]

census_tracts_without_access_m1 = set(no_access_buildings['geo_id'])


#### Model 2

In [27]:
chosen_store ='96061-C'   # Manually taken from model 2 results

chosen_store_index = np.where(comm_buildings_array == chosen_store)[0][0]
chosen_store_index

#New residential buildings that will have access based on commercial building
new_building_with_access = res_comm_access_matrix_half_mile[:,6764]
new_building_with_access = new_building_with_access.nonzero()[0]
new_building_with_access = [i for i in new_building_with_access if i not in existing_access_indices]


# Get IDs of all residential buildings with access
res_buildings_array[new_building_with_access]

new_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[new_building_with_access])]
new_access_buildings

#Census Tracts with New Access to Gracery Stores
census_tracts_with_new_access_m2 = set(new_access_buildings['geo_id'])

original_buildings_without_access = np.where(res_access_array_half_mile==0)[0] #Buildings that dont have access
remaining_buildings_without_access = [i for i in original_buildings_without_access if i not in new_building_with_access]
no_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[remaining_buildings_without_access])]

census_tracts_without_access_m2 = set(no_access_buildings['geo_id'])


#### Baseline model

In [28]:
chosen_store ='106749-C'    # Taken directly from baseline model output

chosen_store_index = np.where(comm_buildings_array == chosen_store)[0][0]

#New residential buildings that will have access based on commercial building
new_building_with_access = res_comm_access_matrix_half_mile[:, chosen_store_index]
new_building_with_access = new_building_with_access.nonzero()[0]
new_building_with_access = [i for i in new_building_with_access if i not in existing_access_indices]

# Get IDs of all residential buildings with access
new_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[new_building_with_access])]

#Census Tracts with New Access to Gracery Stores
census_tracts_with_new_access_b = set(new_access_buildings['geo_id'])

original_buildings_without_access = np.where(res_access_array_half_mile==0)[0] #Buildings that dont have access
remaining_buildings_without_access = [i for i in original_buildings_without_access if i not in new_building_with_access]
no_access_buildings = buildings_df[buildings_df.building_id.isin(res_buildings_array[remaining_buildings_without_access])]

census_tracts_without_access_b = set(no_access_buildings['geo_id'])


## Equity Calculations

#### Calculations Model 1

In [29]:
# CENSUS TRACTS WITH NEW GAINED ACCESS FROM THE NEW STORE PLACEMENT
# Store vars

tracts_new_access = census_tracts_with_new_access_m1
tracts_no_access = census_tracts_without_access_m1

# Equity Calculations 
new_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_new_access)]

# Calculations 
# ACCESS 
sum_pop_access = sum(new_access["USDA_Data_Pop2010"]) 
sum_hh_access = sum(new_access["USDA_Data_OHU2010"]) 

# Population
avg_seniors_access = sum(new_access["Senior"])/sum_pop_access 
avg_black_access = sum(new_access["Black_Pop"])/sum_pop_access 
avg_disability_access = sum(new_access["Disability"])/sum_pop_access 

# Households
avg_SNAP_access = sum(new_access["SNAP"]) / sum_hh_access 
avg_single_access = sum(new_access["SingleHH"]) / sum_hh_access
avg_novehicle_access = sum(new_access["NoVehicle"]) / sum_hh_access
avg_poverty_access = sum(new_access["USDA_Data_OHU2010"]*new_access["USDA_Data_PovertyRate"]/sum_hh_access)

print("CENSUS TRACTS WITH ACCESS")
print("Average % Senior Population:", avg_seniors_access)
print("Average % Black Population:", avg_black_access)
print("Average % Disabled Population:", avg_disability_access)

print("Average % Households that are SNAP Recipients:", avg_SNAP_access)
print("Average % Single Head Households:", avg_single_access)
print("Average % Household with No Vehicle:", avg_novehicle_access)
print("Average Poverty Rate:", avg_poverty_access, "\n")



CENSUS TRACTS WITH ACCESS
Average % Senior Population: 0.1418839250622298
Average % Black Population: 0.31750294772697496
Average % Disabled Population: 0.16219048866762742
Average % Households that are SNAP Recipients: 0.25577507598784194
Average % Single Head Households: 0.1886018237082067
Average % Household with No Vehicle: 0.18419452887537993
Average Poverty Rate: 0.2237737082066869 



In [30]:
# CENSUS TRACTS WITH STILL NO ACCESS EVEN AFTER THE NEW STORE PLACEMENT
no_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_no_access)]

sum_pop_noaccess = sum(no_access["USDA_Data_Pop2010"]) 
sum_hh_noaccess = sum(no_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_noaccess = sum(no_access["Senior"])/sum_pop_noaccess
avg_black_noaccess = sum(no_access["Black_Pop"])/sum_pop_noaccess 
avg_disability_noaccess = sum(no_access["Disability"])/sum_pop_noaccess

#Households
avg_SNAP_noaccess = sum(no_access["SNAP"]) / sum_hh_noaccess
avg_single_noaccess = sum(no_access["SingleHH"]) / sum_hh_noaccess
avg_novehicle_noaccess = sum(no_access["NoVehicle"]) / sum_hh_noaccess
avg_poverty_noaccess = sum(no_access["USDA_Data_OHU2010"]*no_access["USDA_Data_PovertyRate"]/sum_hh_noaccess)

print("CENSUS TRACTS WITH NO ACCESS")
print("Average % Senior Population:", avg_seniors_noaccess)
print("Average % Black Population:", avg_black_noaccess)
print("Average % Disabled Population:", avg_disability_noaccess)

print("Average % Households that are SNAP Recipients:", avg_SNAP_noaccess)
print("Average % Single Head Households:", avg_single_noaccess)
print("Average % Household with No Vehicle:", avg_novehicle_noaccess)
print("Average Poverty Rate:", avg_poverty_noaccess, "\n")



CENSUS TRACTS WITH NO ACCESS
Average % Senior Population: 0.16257623523172565
Average % Black Population: 0.1749977902831384
Average % Disabled Population: 0.14374355499248695
Average % Households that are SNAP Recipients: 0.13939981943377666
Average % Single Head Households: 0.09409656250421096
Average % Household with No Vehicle: 0.16556844672622656
Average Poverty Rate: 0.17609987737666924 



#### Calculations Model 2

In [31]:
# CENSUS TRACTS WITH NEWLY GAINED ACCESS

tracts_new_access = census_tracts_with_new_access_m2
tracts_no_access = census_tracts_without_access_m2

#Equity Calculations 
new_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_new_access)]
no_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_no_access)]

#ACCESS 
sum_pop_access = sum(new_access["USDA_Data_Pop2010"]) 
sum_hh_access = sum(new_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_access = sum(new_access["Senior"])/sum_pop_access 
avg_black_access = sum(new_access["Black_Pop"])/sum_pop_access 
avg_disability_access = sum(new_access["Disability"])/sum_pop_access 

#Households
avg_SNAP_access = sum(new_access["SNAP"]) / sum_hh_access 
avg_single_access = sum(new_access["SingleHH"]) / sum_hh_access
avg_novehicle_access = sum(new_access["NoVehicle"]) / sum_hh_access
avg_poverty_access = sum(new_access["USDA_Data_OHU2010"]*new_access["USDA_Data_PovertyRate"]/sum_hh_access)

print("CENSUS TRACTS WITH ACCESS")
print("Average % Senior Population:", avg_seniors_access)
print("Average % Black Population:", avg_black_access)
print("Average % Disabled Population:", avg_disability_access)

print("Average % Households that are SNAP Recipients:", avg_SNAP_access)
print("Average % Single Head Households:", avg_single_access)
print("Average % Household with No Vehicle:", avg_novehicle_access)
print("Average Poverty Rate:", avg_poverty_access, "\n")

CENSUS TRACTS WITH ACCESS
Average % Senior Population: 0.1353301886792453
Average % Black Population: 0.13547169811320756
Average % Disabled Population: 0.14221698113207548
Average % Households that are SNAP Recipients: 0.11842712177121771
Average % Single Head Households: 0.047624538745387454
Average % Household with No Vehicle: 0.2456180811808118
Average Poverty Rate: 0.2252512684501845 



In [32]:
# CENSUS TRACTS STILL LACKING ACCESS

sum_pop_noaccess = sum(no_access["USDA_Data_Pop2010"]) 
sum_hh_noaccess = sum(no_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_noaccess = sum(no_access["Senior"])/sum_pop_noaccess
avg_black_noaccess = sum(no_access["Black_Pop"])/sum_pop_noaccess 
avg_disability_noaccess = sum(no_access["Disability"])/sum_pop_noaccess

#Households
avg_SNAP_noaccess = sum(no_access["SNAP"]) / sum_hh_noaccess
avg_single_noaccess = sum(no_access["SingleHH"]) / sum_hh_noaccess
avg_novehicle_noaccess = sum(no_access["NoVehicle"]) / sum_hh_noaccess
avg_poverty_noaccess = sum(no_access["USDA_Data_OHU2010"]*no_access["USDA_Data_PovertyRate"]/sum_hh_noaccess)

print("CENSUS TRACTS WITH NO ACCESS")
print("Average % Senior Population:", avg_seniors_noaccess)
print("Average % Black Population:", avg_black_noaccess)
print("Average % Disabled Population:", avg_disability_noaccess)

print("Average % Households that are SNAP Recipients:", avg_SNAP_noaccess)
print("Average % Single Head Households:", avg_single_noaccess)
print("Average % Household with No Vehicle:", avg_novehicle_noaccess)
print("Average Poverty Rate:", avg_poverty_noaccess, "\n")

CENSUS TRACTS WITH NO ACCESS
Average % Senior Population: 0.1655660822762897
Average % Black Population: 0.18642787447672407
Average % Disabled Population: 0.1452330741871518
Average % Households that are SNAP Recipients: 0.14445625221395678
Average % Single Head Households: 0.09970952886999646
Average % Household with No Vehicle: 0.1675522493800921
Average Poverty Rate: 0.18067285866099883 



#### Calculations Baseline

In [33]:
# CENSUS TRACTS WITH NEWLY GAINED ACCESS
tracts_new_access = census_tracts_with_new_access_b
tracts_no_access = census_tracts_without_access_b

# Equity Calculations 
new_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_new_access)]
no_access = merged[merged.Allegheny_Tracts_AFFGEOID.isin(tracts_no_access)]

#Calculations 
#ACCESS 
sum_pop_access = sum(new_access["USDA_Data_Pop2010"]) 
sum_hh_access = sum(new_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_access = sum(new_access["Senior"])/sum_pop_access 
avg_black_access = sum(new_access["Black_Pop"])/sum_pop_access 
avg_disability_access = sum(new_access["Disability"])/sum_pop_access 

#Households
avg_SNAP_access = sum(new_access["SNAP"]) / sum_hh_access 
avg_single_access = sum(new_access["SingleHH"]) / sum_hh_access
avg_novehicle_access = sum(new_access["NoVehicle"]) / sum_hh_access
avg_poverty_access = sum(new_access["USDA_Data_OHU2010"]*new_access["USDA_Data_PovertyRate"]/sum_hh_access)

print("CENSUS TRACTS WITH ACCESS")
print("Average % Senior Population:", avg_seniors_access)
print("Average % Black Population:", avg_black_access)
print("Average % Disabled Population:", avg_disability_access)

print("Average % Households that are SNAP Recipients:", avg_SNAP_access)
print("Average % Single Head Households:", avg_single_access)
print("Average % Household with No Vehicle:", avg_novehicle_access)
print("Average Poverty Rate:", avg_poverty_access, "\n")


CENSUS TRACTS WITH ACCESS
Average % Senior Population: 0.12062561094819159
Average % Black Population: 0.2305962854349951
Average % Disabled Population: 0.1349462365591398
Average % Households that are SNAP Recipients: 0.16016898008449004
Average % Single Head Households: 0.07097163548581774
Average % Household with No Vehicle: 0.2595051297525649
Average Poverty Rate: 0.3614235365117683 



In [34]:
# CENSUS TRACTS WITH STILL NO ACCESS
sum_pop_noaccess = sum(no_access["USDA_Data_Pop2010"]) 
sum_hh_noaccess = sum(no_access["USDA_Data_OHU2010"]) 

#Population
avg_seniors_noaccess = sum(no_access["Senior"])/sum_pop_noaccess
avg_black_noaccess = sum(no_access["Black_Pop"])/sum_pop_noaccess 
avg_disability_noaccess = sum(no_access["Disability"])/sum_pop_noaccess

#Households
avg_SNAP_noaccess = sum(no_access["SNAP"]) / sum_hh_noaccess
avg_single_noaccess = sum(no_access["SingleHH"]) / sum_hh_noaccess
avg_novehicle_noaccess = sum(no_access["NoVehicle"]) / sum_hh_noaccess
avg_poverty_noaccess = sum(no_access["USDA_Data_OHU2010"]*no_access["USDA_Data_PovertyRate"]/sum_hh_noaccess)

print("CENSUS TRACTS WITH NO ACCESS")
print("Average % Senior Population:", avg_seniors_noaccess)
print("Average % Black Population:", avg_black_noaccess)
print("Average % Disabled Population:", avg_disability_noaccess)

print("Average % Households that are SNAP Recipients:", avg_SNAP_noaccess)
print("Average % Single Head Households:", avg_single_noaccess)
print("Average % Household with No Vehicle:", avg_novehicle_noaccess)
print("Average Poverty Rate:", avg_poverty_noaccess, "\n")

CENSUS TRACTS WITH NO ACCESS
Average % Senior Population: 0.16774234132226207
Average % Black Population: 0.18074479461533624
Average % Disabled Population: 0.14914763791910424
Average % Households that are SNAP Recipients: 0.1417903031163484
Average % Single Head Households: 0.09801945055725136
Average % Household with No Vehicle: 0.1612834528288493
Average Poverty Rate: 0.1565234471498544 

